# Importing the Libraries

In [ ]:
import collections.abc
import collections
import cv2
import json
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import pycocotools
import pylab
import re
import shutil
import tensorflow as tf
import time as time
import wget
import zipfile

from cv2 import dnn_superres
from dict2xml import dict2xml
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from PIL import Image
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from time import time

from Cluster import *
from Optimizer import *
from OSCOD import *
from Utils import *

In [ ]:
#PLEASE DEFINE WHERE DO YOU WANT TO CREATE THE FOLDER TO SAVE THE SR MODEL
OUTPUT_DIR = ''
create_dir(OUTPUT_DIR)

# Download Super-Resolution Model

In [ ]:
MODEL_NAME = 'FSRCNN_x2'
MODEL_SR_DIR = OUTPUT_DIR+'MODEL_SR'
create_dir(MODEL_SR_DIR)
PATH_TO_MODEL_DIR = download_model_SR(MODEL_NAME, MODEL_SR_DIR+'/FSRCNN_x2.pb')

100% [..........................................................] 38973 / 38973

# Download Object Detection Model - EfficientDet D4

The presented proposal can be executed with any model provided in Tensorflow Model Zoo. 

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In this example, we are going to use EfficientDet D4 since it is one of the best performing models.

In [ ]:
MODEL_DATE = '20200711'
MODEL_NAME = 'efficientdet_d4_coco17_tpu-32'
PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)

# Download the labels

In [ ]:
LABEL_FILENAME = 'mscoco_complete_label_map.pbtxt'
PATH_TO_LABELS = download_labels(LABEL_FILENAME)

# Loading the Object Detection Model

In [ ]:
import time
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
print('Loading model...', end='')
start_time = time.time()
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...

Done! Took 39.44588780403137 seconds


# Loading Labels

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Start Our Proposal

In [ ]:
matplotlib.use('Agg')

In [ ]:
dict_COCO_names = {
    1:"person", 2:"bicycle", 3:"car", 4:"motorcycle", 5:"airplane", 6:"bus", 7:"train", 8:"truck", 9:"boat", 10:"traffic light", 11:"fire hydrant", 12:"street sign", 13:"stop sign", 14:"parking meter", 15:"bench", 16:"bird", 17:"cat", 18:"dog", 19:"horse", 20:"sheep", 21:"cow", 22:"elephant", 23:"bear", 24:"zebra", 25:"giraffe", 26:"hat", 27:"backpack", 28:"umbrella", 29:"shoe", 30:"eye glasses", 31:"handbag", 32:"tie", 33:"suitcase", 34:"frisbee", 35:"skis", 36:"snowboard", 37:"sports ball", 38:"kite", 39:"baseball bat", 40:"baseball glove", 41:"skateboard", 42:"surfboard", 43:"tennis racket", 44:"bottle", 45:"plate", 46:"wine glass", 47:"cup", 48:"fork", 49:"knife", 50:"spoon", 51:"bowl", 52:"banana", 53:"apple", 54:"sandwich", 55:"orange", 56:"broccoli", 57:"carrot", 58:"hot dog", 59:"pizza", 60:"donut", 61:"cake", 62:"chair", 63:"couch", 64:"potted plant", 65:"bed", 66:"mirror", 67:"dining table", 68:"window", 69:"desk", 70:"toilet", 71:"door", 72:"tv", 73:"laptop", 74:"mouse", 75:"remote", 76:"keyboard", 77:"cell phone", 78:"microwave", 79:"oven", 80:"toaster", 81:"sink", 82:"refrigerator", 83:"blender", 84:"book", 85:"clock", 86:"vase", 87:"scissors", 88:"teddy bear", 89:"hair drier", 90:"toothbrush", 91:"hair brush"
}

In [ ]:
#Arguments: make_inference_SRSR(model_loaded,category_index,path_image_infer,name_image,path_save_image_with_detections,clases_to_detect,% size of sub-imagen,% size of windows Size R,min_score_detect_element,path_model_sr)

#Arguments: 


In [ ]:
'''
make_inference_SRSR(
  model_loaded, 
  category_index,
  path_image_infer,
  name_image,
  path_save_image_with_detections,
  clases_to_detect,
  % size of sub-imagen,
  % size of windows Size R,
  min_score_detect_element,
  path_model_sr
)
'''

In [ ]:
make_inference_SRSR(detect_fn,category_index,'/opt/share/Github/TEST/0.jpg','salida.jpg','/opt/share/Github/TEST/PRUEBA',[3],0.5,0,0.35,'/opt/share/Github/TEST/MODEL_SR/')

# Other proposals presented in the paper.

In [ ]:
make_inference_SRSR(detect_fn,category_index,'/opt/share/ES/0000213_03920_d_0000243.jpg','salidav2ours.jpg','/opt/share/Github/TEST/PRUEBA',[3],0.5,0.5,0.5,'/opt/share/Github/TEST/MODEL_SR/')

In [ ]:
make_inference_SRSRRAW(detect_fn,category_index,'/opt/share/ES/0000213_03920_d_0000243.jpg','salidav2raw.jpg','/opt/share/Github/TEST/PRUEBA',[3],0.5)

# Evaluation with different models and Windows Size R:

In [ ]:
%%capture

import json
import time

#PLEASE INSERT THE PATH TO GENERATE THE OUTPUTS
PATH_EVAL = ''
    
#PLEASE INSERT THE PATH OF EVERY SEQUENCE
SEQ_DATA_VAL = ['/opt/share/DATA-CLICK/NGSIM-Dataset-Annotations-main/Sequence1/images/test/','/opt/share/DATA-CLICK/NGSIM-Dataset-Annotations-main/Sequence2/images/test/','/opt/share/DATA-CLICK/NGSIM-Dataset-Annotations-main/Sequence3/images/test/','/opt/share/DATA-CLICK/NGSIM-Dataset-Annotations-main/Sequence4/images/test/']
SEQ_DATA_NAME = ['SEQ1','SEQ2','SEQ3','SEQ4']
SEQ_DATA_GT = ['/opt/share/DATA-CLICK/NGSIM-Dataset-Annotations-main/Sequence1/annotations/GT.json','/opt/share/DATA-CLICK/NGSIM-Dataset-Annotations-main/Sequence2/annotations/GT.json','/opt/share/DATA-CLICK/NGSIM-Dataset-Annotations-main/Sequence3/annotations/GT.json','/opt/share/DATA-CLICK/NGSIM-Dataset-Annotations-main/Sequence4/annotations/GT.json',]

#SELECTION OF DIFFERENT PRE-TRAINED MODELS
MODELS_P1 = ['20200711','20200711','20200711','20200711','20200711','20200711']
MODELS_P2 = ['centernet_hg104_1024x1024_kpts_coco17_tpu-32','centernet_resnet101_v1_fpn_512x512_coco17_tpu-8','faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8','efficientdet_d3_coco17_tpu-32','efficientdet_d4_coco17_tpu-32','efficientdet_d5_coco17_tpu-32']
MODELS_NAMES = ['CENTERNET-HG104KPT1024','CENTERNET-RESNET101-V1','FASTER-INCEPTION-RESNET-V2-1024','ED3','ED4','ED5']

#PLEASE INSERT THE CLASSES (COCO) THAT YOU WANT TO DETECT
CLASES = [3]
        
#PLEASE INSERT THE MINIMUN SCORE TO DETECT ONE OBJECT
SCORE = 0.35

#PLEASE INSERT % OF THE WINDOWS SIZE R TO BE EVALLUATED
WSizeR = [0, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1]

for modelo_selected in range(len(MODELS_P1)):
    MODEL_DATE = MODELS_P1[modelo_selected]
    MODEL_NAME = MODELS_P2[modelo_selected]

    PATH_TO_MODEL_DIR = download_model(MODEL_NAME, MODEL_DATE)
    LABEL_FILENAME = 'mscoco_complete_label_map.pbtxt'
    PATH_TO_LABELS = download_labels(LABEL_FILENAME)
    PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
    print('Loading model...', end='')
    start_time = time.time()
    detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Done! Took {} seconds'.format(elapsed_time))

    category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

    for seq_selected in range(len(SEQ_DATA_VAL)):
        
        jsonString = SEQ_DATA_GT[seq_selected]
        dict_ids = {}

        with open(jsonString) as json_file:
            data = json.load(json_file)
            for p3 in data['images']:
                data_s_image = p3['file_name'].replace('images/','')
                data_s_id = p3['id']
                dict_ids[data_s_image] = data_s_id
                
        imagenes_dir_train = SEQ_DATA_VAL[seq_selected]

        for parte in range(len(WSizeR)):

            MODEL = MODELS_NAMES[modelo_selected]+'/'+SEQ_DATA_NAME[seq_selected]
            MODEL = MODEL+'/'+str(WSizeR[parte])
            test_dataset = os.listdir(imagenes_dir_train)

            create_dir(PATH_EVAL+MODEL)
            create_dir(PATH_EVAL+MODEL+'/FACTOR-OPT/SALIDA/')
            create_dir(PATH_EVAL+MODEL+'/AUX/')
            create_dir(PATH_EVAL+MODEL+'/JSON/')
            PATH_AUX = PATH_EVAL+MODEL+'/AUX/'

            OPTIMIZATION = WSizeR[parte]
            WINDOW_SIZE  = 0.50

            imagenes_dir3 = imagenes_dir_train
            frames_test = os.listdir(imagenes_dir_train)
            imagenes_dir2 = []
            counter = 0

            result = []
            result2 = []
            resultaux = []
            out = []
            out2 = []
            outaux = []

            for frame in frames_test:
              id = frame.replace('.jpg','')
              image_path_save = imagenes_dir_train+"/"+str(frame)
              output = make_inference_SRSR(detect_fn,category_index,image_path_save,frame,PATH_EVAL+'/'+MODEL+'/FACTOR-OPT/',CLASES,WINDOW_SIZE,OPTIMIZATION,SCORE,MODEL_SR_DIR+'/')   
              counter = counter+1

              result = []
              result2 = []
              resultaux = []
              out2= []
              converted_num = dict_ids[frame]
              width = output[3]
              height = output[4]
              boxes2 = []

              for box in output[0]:
                ymin = int(box[0]*height)
                xmin = int(box[1]*width)
                ymax = int(box[2]*height)
                xmax = int(box[3]*width) 
                box_new = []
                box_new.append(xmin)
                box_new.append(ymin)
                box_new.append(xmax-xmin)
                box_new.append(ymax-ymin)
                boxes2.append(box_new)

              result.extend(
                    [
                        {
                            "image_id": converted_num,
                            "category_id": int(output[2][k]),
                            "bbox": box,
                            "score": output[1][k].astype(float),
                        }
                        for k, box in enumerate(boxes2)
                    ]
              )

              result2.extend(
                    [
                        {
                            "image_id": converted_num,
                            "category_id": int(output[2][k]),
                            "bbox": box,
                            "score": output[1][k].astype(float),
                        }
                        for k, box in enumerate(boxes2)
                    ]
              )

              cluster_converted = []
              raw_subimages = output[6]
              ours_subimages = output[7]

              for indexcluster in range(len(output[5])):
                    tripla = output[5][indexcluster]
                    lista_boxes_lista = tripla[0].tolist()
                    lista_clases_lista = tripla[1].tolist()
                    lista_scores_lista = tripla[2].tolist()
                    cluster_aux = []
                    cluster_aux.append(lista_boxes_lista)
                    cluster_aux.append(lista_clases_lista)
                    cluster_aux.append(lista_scores_lista)
                    cluster_converted.append(cluster_aux)

              resultaux.extend(
                [
                    {
                        "image_id": converted_num,
                        "triplas": cluster_converted,
                        "number_subimages": int(ours_subimages),
                        "number_subimages_without_proposal":int(raw_subimages),
                    }
                ]
              )


              for save_file in result2:    
                    out2.append(save_file)

              json_file = PATH_EVAL+MODEL+'/JSON/'+str(converted_num)+'.json'
              with open(json_file, 'w', encoding='utf-8') as save_files:
                json.dump(out2, save_files, ensure_ascii=False)

              for sample in result:    
                    out.append(sample)

              for sampleaux in resultaux:    
                    outaux.append(sampleaux)

            with open(PATH_EVAL+MODEL+'/'+'test_data_modificado.json', 'w', encoding='utf-8') as file:
              json.dump(out, file, ensure_ascii=False)

            with open(PATH_EVAL+MODEL+'/'+'test_data_modificado2.json', 'w', encoding='utf-8') as fileaux:
              json.dump(outaux, fileaux, ensure_ascii=False)

In [ ]:
#Please insert a path to automatize the saving of data and executing the following cell
path_auto = ''

In [ ]:
__author__ = 'tsungyi'

import numpy as np
import datetime
import time
from collections import defaultdict
import mask as maskUtils
import copy
import json

class COCOeval:
    # Interface for evaluating detection on the Microsoft COCO dataset.
    #
    # The usage for CocoEval is as follows:
    #  cocoGt=..., cocoDt=...       # load dataset and results
    #  E = CocoEval(cocoGt,cocoDt); # initialize CocoEval object
    #  E.params.recThrs = ...;      # set parameters as desired
    #  E.evaluate();                # run per image evaluation
    #  E.accumulate();              # accumulate per image results
    #  E.summarize();               # display summary metrics of results
    # For example usage see evalDemo.m and http://mscoco.org/.
    #
    # The evaluation parameters are as follows (defaults in brackets):
    #  imgIds     - [all] N img ids to use for evaluation
    #  catIds     - [all] K cat ids to use for evaluation
    #  iouThrs    - [.5:.05:.95] T=10 IoU thresholds for evaluation
    #  recThrs    - [0:.01:1] R=101 recall thresholds for evaluation
    #  areaRng    - [...] A=4 object area ranges for evaluation
    #  maxDets    - [1 10 100] M=3 thresholds on max detections per image
    #  iouType    - ['segm'] set iouType to 'segm', 'bbox' or 'keypoints'
    #  iouType replaced the now DEPRECATED useSegm parameter.
    #  useCats    - [1] if true use category labels for evaluation
    # Note: if useCats=0 category labels are ignored as in proposal scoring.
    # Note: multiple areaRngs [Ax2] and maxDets [Mx1] can be specified.
    #
    # evaluate(): evaluates detections on every image and every category and
    # concats the results into the "evalImgs" with fields:
    #  dtIds      - [1xD] id for each of the D detections (dt)
    #  gtIds      - [1xG] id for each of the G ground truths (gt)
    #  dtMatches  - [TxD] matching gt id at each IoU or 0
    #  gtMatches  - [TxG] matching dt id at each IoU or 0
    #  dtScores   - [1xD] confidence of each dt
    #  gtIgnore   - [1xG] ignore flag for each gt
    #  dtIgnore   - [TxD] ignore flag for each dt at each IoU
    #
    # accumulate(): accumulates the per-image, per-category evaluation
    # results in "evalImgs" into the dictionary "eval" with fields:
    #  params     - parameters used for evaluation
    #  date       - date evaluation was performed
    #  counts     - [T,R,K,A,M] parameter dimensions (see above)
    #  precision  - [TxRxKxAxM] precision for every evaluation setting
    #  recall     - [TxKxAxM] max recall for every evaluation setting
    # Note: precision and recall==-1 for settings with no gt objects.
    #
    # See also coco, mask, pycocoDemo, pycocoEvalDemo
    #
    # Microsoft COCO Toolbox.      version 2.0
    # Data, paper, and tutorials available at:  http://mscoco.org/
    # Code written by Piotr Dollar and Tsung-Yi Lin, 2015.
    # Licensed under the Simplified BSD License [see coco/license.txt]
    def __init__(self, cocoGt=None, cocoDt=None, iouType='segm'):
        '''
        Initialize CocoEval using coco APIs for gt and dt
        :param cocoGt: coco object with ground truth annotations
        :param cocoDt: coco object with detection results
        :return: None
        '''
        if not iouType:
            print('iouType not specified. use default iouType segm')
        self.cocoGt   = cocoGt              # ground truth COCO API
        self.cocoDt   = cocoDt              # detections COCO API
        self.evalImgs = defaultdict(list)   # per-image per-category evaluation results [KxAxI] elements
        self.eval     = {}                  # accumulated evaluation results
        self._gts = defaultdict(list)       # gt for evaluation
        self._dts = defaultdict(list)       # dt for evaluation
        self.params = Params(iouType=iouType) # parameters
        self._paramsEval = {}               # parameters for evaluation
        self.stats = []                     # result summarization
        self.ious = {}                      # ious between all gts and dts
        if not cocoGt is None:
            self.params.imgIds = sorted(cocoGt.getImgIds())
            self.params.catIds = sorted(cocoGt.getCatIds())


    def _prepare(self):
        '''
        Prepare ._gts and ._dts for evaluation based on params
        :return: None
        '''
        def _toMask(anns, coco):
            # modify ann['segmentation'] by reference
            for ann in anns:
                rle = coco.annToRLE(ann)
                ann['segmentation'] = rle
        p = self.params
        if p.useCats:
            gts=self.cocoGt.loadAnns(self.cocoGt.getAnnIds(imgIds=p.imgIds, catIds=p.catIds))
            dts=self.cocoDt.loadAnns(self.cocoDt.getAnnIds(imgIds=p.imgIds, catIds=p.catIds))
        else:
            gts=self.cocoGt.loadAnns(self.cocoGt.getAnnIds(imgIds=p.imgIds))
            dts=self.cocoDt.loadAnns(self.cocoDt.getAnnIds(imgIds=p.imgIds))

        # convert ground truth to mask if iouType == 'segm'
        if p.iouType == 'segm':
            _toMask(gts, self.cocoGt)
            _toMask(dts, self.cocoDt)
        # set ignore flag
        for gt in gts:
            gt['ignore'] = gt['ignore'] if 'ignore' in gt else 0
            gt['ignore'] = 'iscrowd' in gt and gt['iscrowd']
            if p.iouType == 'keypoints':
                gt['ignore'] = (gt['num_keypoints'] == 0) or gt['ignore']
        self._gts = defaultdict(list)       # gt for evaluation
        self._dts = defaultdict(list)       # dt for evaluation
        for gt in gts:
            self._gts[gt['image_id'], gt['category_id']].append(gt)
        for dt in dts:
            self._dts[dt['image_id'], dt['category_id']].append(dt)
        self.evalImgs = defaultdict(list)   # per-image per-category evaluation results
        self.eval     = {}                  # accumulated evaluation results

    def evaluate(self):
        '''
        Run per image evaluation on given images and store results (a list of dict) in self.evalImgs
        :return: None
        '''
        tic = time.time()
        print('Running per image evaluation...')
        p = self.params
        # add backward compatibility if useSegm is specified in params
        if not p.useSegm is None:
            p.iouType = 'segm' if p.useSegm == 1 else 'bbox'
            print('useSegm (deprecated) is not None. Running {} evaluation'.format(p.iouType))
        print('Evaluate annotation type *{}*'.format(p.iouType))
        p.imgIds = list(np.unique(p.imgIds))
        if p.useCats:
            p.catIds = list(np.unique(p.catIds))
        p.maxDets = sorted(p.maxDets)
        self.params=p

        self._prepare()
        # loop through images, area range, max detection number
        catIds = p.catIds if p.useCats else [-1]

        if p.iouType == 'segm' or p.iouType == 'bbox':
            computeIoU = self.computeIoU
        elif p.iouType == 'keypoints':
            computeIoU = self.computeOks
        self.ious = {(imgId, catId): computeIoU(imgId, catId) \
                        for imgId in p.imgIds
                        for catId in catIds}

        evaluateImg = self.evaluateImg
        maxDet = p.maxDets[-1]
        self.evalImgs = [evaluateImg(imgId, catId, areaRng, maxDet)
                 for catId in catIds
                 for areaRng in p.areaRng
                 for imgId in p.imgIds
             ]
        self._paramsEval = copy.deepcopy(self.params)
        toc = time.time()
        print('DONE (t={:0.2f}s).'.format(toc-tic))

    def computeIoU(self, imgId, catId):
        p = self.params
        if p.useCats:
            gt = self._gts[imgId,catId]
            dt = self._dts[imgId,catId]
        else:
            gt = [_ for cId in p.catIds for _ in self._gts[imgId,cId]]
            dt = [_ for cId in p.catIds for _ in self._dts[imgId,cId]]
        if len(gt) == 0 and len(dt) ==0:
            return []
        inds = np.argsort([-d['score'] for d in dt], kind='mergesort')
        dt = [dt[i] for i in inds]
        if len(dt) > p.maxDets[-1]:
            dt=dt[0:p.maxDets[-1]]

        if p.iouType == 'segm':
            g = [g['segmentation'] for g in gt]
            d = [d['segmentation'] for d in dt]
        elif p.iouType == 'bbox':
            g = [g['bbox'] for g in gt]
            d = [d['bbox'] for d in dt]
        else:
            raise Exception('unknown iouType for iou computation')

        # compute iou between each dt and gt region
        iscrowd = [int(o['iscrowd']) for o in gt]
        ious = maskUtils.iou(d,g,iscrowd)
        return ious

    def computeOks(self, imgId, catId):
        p = self.params
        # dimention here should be Nxm
        gts = self._gts[imgId, catId]
        dts = self._dts[imgId, catId]
        inds = np.argsort([-d['score'] for d in dts], kind='mergesort')
        dts = [dts[i] for i in inds]
        if len(dts) > p.maxDets[-1]:
            dts = dts[0:p.maxDets[-1]]
        # if len(gts) == 0 and len(dts) == 0:
        if len(gts) == 0 or len(dts) == 0:
            return []
        ious = np.zeros((len(dts), len(gts)))
        sigmas = p.kpt_oks_sigmas
        vars = (sigmas * 2)**2
        k = len(sigmas)
        # compute oks between each detection and ground truth object
        for j, gt in enumerate(gts):
            # create bounds for ignore regions(double the gt bbox)
            g = np.array(gt['keypoints'])
            xg = g[0::3]; yg = g[1::3]; vg = g[2::3]
            k1 = np.count_nonzero(vg > 0)
            bb = gt['bbox']
            x0 = bb[0] - bb[2]; x1 = bb[0] + bb[2] * 2
            y0 = bb[1] - bb[3]; y1 = bb[1] + bb[3] * 2
            for i, dt in enumerate(dts):
                d = np.array(dt['keypoints'])
                xd = d[0::3]; yd = d[1::3]
                if k1>0:
                    # measure the per-keypoint distance if keypoints visible
                    dx = xd - xg
                    dy = yd - yg
                else:
                    # measure minimum distance to keypoints in (x0,y0) & (x1,y1)
                    z = np.zeros((k))
                    dx = np.max((z, x0-xd),axis=0)+np.max((z, xd-x1),axis=0)
                    dy = np.max((z, y0-yd),axis=0)+np.max((z, yd-y1),axis=0)
                e = (dx**2 + dy**2) / vars / (gt['area']+np.spacing(1)) / 2
                if k1 > 0:
                    e=e[vg > 0]
                ious[i, j] = np.sum(np.exp(-e)) / e.shape[0]
        return ious

    def evaluateImg(self, imgId, catId, aRng, maxDet):
        '''
        perform evaluation for single category and image
        :return: dict (single image results)
        '''
        p = self.params
        if p.useCats:
            gt = self._gts[imgId,catId]
            dt = self._dts[imgId,catId]
        else:
            gt = [_ for cId in p.catIds for _ in self._gts[imgId,cId]]
            dt = [_ for cId in p.catIds for _ in self._dts[imgId,cId]]
        if len(gt) == 0 and len(dt) ==0:
            return None

        for g in gt:
            if g['ignore'] or (g['area']<aRng[0] or g['area']>aRng[1]):
                g['_ignore'] = 1
            else:
                g['_ignore'] = 0

        # sort dt highest score first, sort gt ignore last
        gtind = np.argsort([g['_ignore'] for g in gt], kind='mergesort')
        gt = [gt[i] for i in gtind]
        dtind = np.argsort([-d['score'] for d in dt], kind='mergesort')
        dt = [dt[i] for i in dtind[0:maxDet]]
        iscrowd = [int(o['iscrowd']) for o in gt]
        # load computed ious
        ious = self.ious[imgId, catId][:, gtind] if len(self.ious[imgId, catId]) > 0 else self.ious[imgId, catId]

        T = len(p.iouThrs)
        G = len(gt)
        D = len(dt)
        gtm  = np.zeros((T,G))
        dtm  = np.zeros((T,D))
        gtIg = np.array([g['_ignore'] for g in gt])
        dtIg = np.zeros((T,D))
        if not len(ious)==0:
            for tind, t in enumerate(p.iouThrs):
                for dind, d in enumerate(dt):
                    # information about best match so far (m=-1 -> unmatched)
                    iou = min([t,1-1e-10])
                    m   = -1
                    for gind, g in enumerate(gt):
                        # if this gt already matched, and not a crowd, continue
                        if gtm[tind,gind]>0 and not iscrowd[gind]:
                            continue
                        # if dt matched to reg gt, and on ignore gt, stop
                        if m>-1 and gtIg[m]==0 and gtIg[gind]==1:
                            break
                        # continue to next gt unless better match made
                        if ious[dind,gind] < iou:
                            continue
                        # if match successful and best so far, store appropriately
                        iou=ious[dind,gind]
                        m=gind
                    # if match made store id of match for both dt and gt
                    if m ==-1:
                        continue
                    dtIg[tind,dind] = gtIg[m]
                    dtm[tind,dind]  = gt[m]['id']
                    gtm[tind,m]     = d['id']
        # set unmatched detections outside of area range to ignore
        a = np.array([d['area']<aRng[0] or d['area']>aRng[1] for d in dt]).reshape((1, len(dt)))
        dtIg = np.logical_or(dtIg, np.logical_and(dtm==0, np.repeat(a,T,0)))
        # store results for given image and category
        return {
                'image_id':     imgId,
                'category_id':  catId,
                'aRng':         aRng,
                'maxDet':       maxDet,
                'dtIds':        [d['id'] for d in dt],
                'gtIds':        [g['id'] for g in gt],
                'dtMatches':    dtm,
                'gtMatches':    gtm,
                'dtScores':     [d['score'] for d in dt],
                'gtIgnore':     gtIg,
                'dtIgnore':     dtIg,
            }

    def accumulate(self, p = None):
        '''
        Accumulate per image evaluation results and store the result in self.eval
        :param p: input params for evaluation
        :return: None
        '''
        print('Accumulating evaluation results...')
        tic = time.time()
        if not self.evalImgs:
            print('Please run evaluate() first')
        # allows input customized parameters
        if p is None:
            p = self.params
        p.catIds = p.catIds if p.useCats == 1 else [-1]
        T           = len(p.iouThrs)
        R           = len(p.recThrs)
        K           = len(p.catIds) if p.useCats else 1
        A           = len(p.areaRng)
        M           = len(p.maxDets)
        precision   = -np.ones((T,R,K,A,M)) # -1 for the precision of absent categories
        recall      = -np.ones((T,K,A,M))
        scores      = -np.ones((T,R,K,A,M))

        # create dictionary for future indexing
        _pe = self._paramsEval
        catIds = _pe.catIds if _pe.useCats else [-1]
        setK = set(catIds)
        setA = set(map(tuple, _pe.areaRng))
        setM = set(_pe.maxDets)
        setI = set(_pe.imgIds)
        # get inds to evaluate
        k_list = [n for n, k in enumerate(p.catIds)  if k in setK]
        m_list = [m for n, m in enumerate(p.maxDets) if m in setM]
        a_list = [n for n, a in enumerate(map(lambda x: tuple(x), p.areaRng)) if a in setA]
        i_list = [n for n, i in enumerate(p.imgIds)  if i in setI]
        I0 = len(_pe.imgIds)
        A0 = len(_pe.areaRng)
        # retrieve E at each category, area range, and max number of detections
        for k, k0 in enumerate(k_list):
            Nk = k0*A0*I0
            for a, a0 in enumerate(a_list):
                Na = a0*I0
                for m, maxDet in enumerate(m_list):
                    E = [self.evalImgs[Nk + Na + i] for i in i_list]
                    E = [e for e in E if not e is None]
                    if len(E) == 0:
                        continue
                    dtScores = np.concatenate([e['dtScores'][0:maxDet] for e in E])

                    # different sorting method generates slightly different results.
                    # mergesort is used to be consistent as Matlab implementation.
                    inds = np.argsort(-dtScores, kind='mergesort')
                    dtScoresSorted = dtScores[inds]

                    dtm  = np.concatenate([e['dtMatches'][:,0:maxDet] for e in E], axis=1)[:,inds]
                    dtIg = np.concatenate([e['dtIgnore'][:,0:maxDet]  for e in E], axis=1)[:,inds]
                    gtIg = np.concatenate([e['gtIgnore'] for e in E])
                    npig = np.count_nonzero(gtIg==0 )
                    if npig == 0:
                        continue
                    tps = np.logical_and(               dtm,  np.logical_not(dtIg) )
                    fps = np.logical_and(np.logical_not(dtm), np.logical_not(dtIg) )

                    tp_sum = np.cumsum(tps, axis=1).astype(dtype=np.float)
                    fp_sum = np.cumsum(fps, axis=1).astype(dtype=np.float)
                    for t, (tp, fp) in enumerate(zip(tp_sum, fp_sum)):
                        tp = np.array(tp)
                        fp = np.array(fp)
                        nd = len(tp)
                        rc = tp / npig
                        pr = tp / (fp+tp+np.spacing(1))
                        q  = np.zeros((R,))
                        ss = np.zeros((R,))

                        if nd:
                            recall[t,k,a,m] = rc[-1]
                        else:
                            recall[t,k,a,m] = 0

                        # numpy is slow without cython optimization for accessing elements
                        # use python array gets significant speed improvement
                        pr = pr.tolist(); q = q.tolist()

                        for i in range(nd-1, 0, -1):
                            if pr[i] > pr[i-1]:
                                pr[i-1] = pr[i]

                        inds = np.searchsorted(rc, p.recThrs, side='left')
                        try:
                            for ri, pi in enumerate(inds):
                                q[ri] = pr[pi]
                                ss[ri] = dtScoresSorted[pi]
                        except:
                            pass
                        precision[t,:,k,a,m] = np.array(q)
                        scores[t,:,k,a,m] = np.array(ss)
        self.eval = {
            'params': p,
            'counts': [T, R, K, A, M],
            'date': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'precision': precision,
            'recall':   recall,
            'scores': scores,
        }
        toc = time.time()
        print('DONE (t={:0.2f}s).'.format( toc-tic))

    def summarize(self):
        '''
        Compute and display summary metrics for evaluation results.
        Note this functin can *only* be applied on the default parameter setting
        '''
        def _summarize( ap=1, iouThr=None, areaRng='all', maxDets=100 ):
            p = self.params
            iStr = ' {:<18} {} @[ IoU={:<9} | area={:>6s} | maxDets={:>3d} ] = {:0.3f}'
            titleStr = 'Average Precision' if ap == 1 else 'Average Recall'
            typeStr = '(AP)' if ap==1 else '(AR)'
            iouStr = '{:0.2f}:{:0.2f}'.format(p.iouThrs[0], p.iouThrs[-1]) \
                if iouThr is None else '{:0.2f}'.format(iouThr)

            aind = [i for i, aRng in enumerate(p.areaRngLbl) if aRng == areaRng]
            mind = [i for i, mDet in enumerate(p.maxDets) if mDet == maxDets]
            if ap == 1:
                # dimension of precision: [TxRxKxAxM]
                s = self.eval['precision']
                # IoU
                if iouThr is not None:
                    t = np.where(iouThr == p.iouThrs)[0]
                    s = s[t]
                s = s[:,:,:,aind,mind]
            else:
                # dimension of recall: [TxKxAxM]
                s = self.eval['recall']
                if iouThr is not None:
                    t = np.where(iouThr == p.iouThrs)[0]
                    s = s[t]
                s = s[:,:,aind,mind]
            if len(s[s>-1])==0:
                mean_s = -1
            else:
                mean_s = np.mean(s[s>-1])
            print(iStr.format(titleStr, typeStr, iouStr, areaRng, maxDets, mean_s))
            return mean_s
        def _summarizeDets():
            stats = np.zeros((12,))
            stats[0] = _summarize(1)
            stats[1] = _summarize(1, iouThr=.5, maxDets=self.params.maxDets[2])
            stats[2] = _summarize(1, iouThr=.75, maxDets=self.params.maxDets[2])
            stats[3] = _summarize(1, areaRng='small', maxDets=self.params.maxDets[2])
            stats[4] = _summarize(1, areaRng='medium', maxDets=self.params.maxDets[2])
            stats[5] = _summarize(1, areaRng='large', maxDets=self.params.maxDets[2])
            stats[6] = _summarize(0, maxDets=self.params.maxDets[0])
            stats[7] = _summarize(0, maxDets=self.params.maxDets[1])
            stats[8] = _summarize(0, maxDets=self.params.maxDets[2])
            stats[9] = _summarize(0, areaRng='small', maxDets=self.params.maxDets[2])
            stats[10] = _summarize(0, areaRng='medium', maxDets=self.params.maxDets[2])
            stats[11] = _summarize(0, areaRng='large', maxDets=self.params.maxDets[2])
            
            #print(round(stats[0],3),' & ',round(stats[1],3),' & ',round(stats[2],3),' & ',round(stats[3],3),' & ',round(stats[4],3))
            
            data = { 'results' : [round(stats[0],3),round(stats[1],3),round(stats[2],3),round(stats[3],3),round(stats[4],3)] }
            json_string = json.dumps(data)
            with open(path_auto, 'w') as outfile:
                outfile.write(json_string)
            
            
            return stats
        def _summarizeKps():
            stats = np.zeros((10,))
            stats[0] = _summarize(1, maxDets=20)
            stats[1] = _summarize(1, maxDets=20, iouThr=.5)
            stats[2] = _summarize(1, maxDets=20, iouThr=.75)
            stats[3] = _summarize(1, maxDets=20, areaRng='medium')
            stats[4] = _summarize(1, maxDets=20, areaRng='large')
            stats[5] = _summarize(0, maxDets=20)
            stats[6] = _summarize(0, maxDets=20, iouThr=.5)
            stats[7] = _summarize(0, maxDets=20, iouThr=.75)
            stats[8] = _summarize(0, maxDets=20, areaRng='medium')
            stats[9] = _summarize(0, maxDets=20, areaRng='large')
            return stats
        if not self.eval:
            raise Exception('Please run accumulate() first')
        iouType = self.params.iouType
        if iouType == 'segm' or iouType == 'bbox':
            summarize = _summarizeDets
        elif iouType == 'keypoints':
            summarize = _summarizeKps
        self.stats = summarize()

    def __str__(self):
        self.summarize()

class Params:
    '''
    Params for coco evaluation api
    '''
    def setDetParams(self):
        self.imgIds = []
        self.catIds = []
        # np.arange causes trouble.  the data point on arange is slightly larger than the true value
        self.iouThrs = np.linspace(.5, 0.95, int(np.round((0.95 - .5) / .05)) + 1, endpoint=True)
        self.recThrs = np.linspace(.0, 1.00, int(np.round((1.00 - .0) / .01)) + 1, endpoint=True)
        self.maxDets = [1, 10, 100]
        self.areaRng = [[0 ** 2, 1e5 ** 2], [0 ** 2, 32 ** 2], [32 ** 2, 96 ** 2], [96 ** 2, 1e5 ** 2]]
        self.areaRngLbl = ['all', 'small', 'medium', 'large']
        self.useCats = 1

    def setKpParams(self):
        self.imgIds = []
        self.catIds = []
        # np.arange causes trouble.  the data point on arange is slightly larger than the true value
        self.iouThrs = np.linspace(.5, 0.95, int(np.round((0.95 - .5) / .05)) + 1, endpoint=True)
        self.recThrs = np.linspace(.0, 1.00, int(np.round((1.00 - .0) / .01)) + 1, endpoint=True)
        self.maxDets = [20]
        self.areaRng = [[0 ** 2, 1e5 ** 2], [32 ** 2, 96 ** 2], [96 ** 2, 1e5 ** 2]]
        self.areaRngLbl = ['all', 'medium', 'large']
        self.useCats = 1
        self.kpt_oks_sigmas = np.array([.26, .25, .25, .35, .35, .79, .79, .72, .72, .62,.62, 1.07, 1.07, .87, .87, .89, .89])/10.0

    def __init__(self, iouType='segm'):
        if iouType == 'segm' or iouType == 'bbox':
            self.setDetParams()
        elif iouType == 'keypoints':
            self.setKpParams()
        else:
            raise Exception('iouType not supported')
        self.iouType = iouType
        # useSegm is deprecated
        self.useSegm = None

In [ ]:
data_final = {
    'CENTERNET-HG104KPT1024' : [
        {
            'SEQ1' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ2' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ3' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
        
            'SEQ4' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
            
        }
        
        
    ],
    
    'CENTERNET-RESNET101-V1' : [
        {
            'SEQ1' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ2' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ3' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
        
            'SEQ4' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
            
        }
        
        
    ],
    
    'FASTER-INCEPTION-RESNET-V2-1024' : [
        {
            'SEQ1' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ2' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ3' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
        
            'SEQ4' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
            
        }
        
        
    ],
    
    'ED3' : [
        {
            'SEQ1' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ2' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ3' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
        
            'SEQ4' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
            
        }
        
        
    ],
    
    'ED4' : [
        {
            'SEQ1' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ2' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ3' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
        
            'SEQ4' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
            
        }
        
        
    ],
    
    'ED5' : [
        {
            'SEQ1' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ2' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
            'SEQ3' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
        },
        
        {
        
            'SEQ4' : [
                {'0'   : []},
                {'10'  : []},
                {'20'  : []},
                {'30'  : []},
                {'40'  : []},
                {'50'  : []},
                {'60'  : []},
                {'70'  : []},
                {'80'  : []},
                {'90'  : []},
                {'100' : []}
            ]
            
        }
        
        
    ]
}

In [ ]:
%%capture

import json
import time
import statistics

#PLEASE INSERT THE PATH TO GENERATE THE JSON WITH ALL THE OUTPUTS
PATH_EVAL_JSON = ''
SEQ_DATA_NAMEOK = ['SEQ1','SEQ2','SEQ3','SEQ4']
SEQ_DATA_GTOK = ['/opt/share/CARPETA-DATOS/ANOTACIONESGITHUB/Sequence1/annotations/GT.json','/opt/share/CARPETA-DATOS/ANOTACIONESGITHUB/Sequence2/annotations/GT.json','/opt/share/CARPETA-DATOS/ANOTACIONESGITHUB/Sequence3/annotations/GT.json','/opt/share/CARPETA-DATOS/ANOTACIONESGITHUB/Sequence4/annotations/GT.json']
MODELS_NAMES = ['CENTERNET-HG104KPT1024','CENTERNET-RESNET101-V1','FASTER-INCEPTION-RESNET-V2-1024','ED3','ED4','ED5']
WSizeROK = [0, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1]
classes = [3]
score = 40
#*****************************************************

for model_name in MODELS_NAMES:
    MODELS_NAMES = model_name
    for num_sec in range(len(SEQ_DATA_NAMEOK)):
        SELECTED_SEQ = num_sec
        for num_r in range(len(WSizeROK)):
            WSizeR = WSizeROK[num_r]
            SEQ_DATA_GT = SEQ_DATA_GTOK[SELECTED_SEQ]
            SEQ_DATA_NAME = SEQ_DATA_NAMEOK[SELECTED_SEQ]
            PATH_MAP = PATH_EVAL+MODELS_NAMES+'/'+SEQ_DATA_NAME+'/'+str(WSizeR)+'/test_data_modificado.json'
            PATH_DET = PATH_EVAL+MODELS_NAMES+'/'+SEQ_DATA_NAME+'/'+str(WSizeR)+'/test_data_modificado2.json'
            
            jsonString = SEQ_DATA_GT
            dict_ids = {}
            annFile = SEQ_DATA_GT
            resFile = PATH_MAP

            with open(jsonString) as json_file:
                data = json.load(json_file)
                for p3 in data['images']:
                    data_s_image = p3['file_name'].replace('images/','')
                    data_s_id = p3['id']
                    dict_ids[data_s_image] = data_s_id   

            annType = ['segm','bbox','keypoints']
            annType = annType[1]      #specify type here
            prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
            cocoGt=COCO(annFile)
            cocoDt=cocoGt.loadRes(resFile)
            dts = json.load(open(resFile,'r'))
            imgIds = [imid['image_id'] for imid in dts]
            imgIds = sorted(list(set(imgIds)))
            cocoEval = COCOeval(cocoGt,cocoDt,annType)
            cocoEval.params.imgIds  = imgIds
            cocoEval.params.catIds = classes
            cocoEval.evaluate()
            cocoEval.accumulate()
            cocoEval.summarize()

            f1 = open(path_auto)
            data1 = json.load(f1)
            f1.close()

            f = open(PATH_DET)
            data = json.load(f)

            counter = 0
            media = 0
            list_elements = []

            for i in data:
                counter = counter + 1
                media = media + (int(i['number_subimages']))
                numerador = int(i['number_subimages'])
                list_elements.append(numerador)

            f.close()

            st_dev = statistics.pstdev(list_elements)
            json_data = []
            for salida in data1['results']:
              json_data.append(salida)

            json_data.append(round(media/counter,3))
            json_data.append(round(st_dev,3))
            
            jsonString = SEQ_DATA_GT
            dict_counter = {}

            with open(jsonString) as json_file:
                data = json.load(json_file)
                for p3 in data['annotations']:
                    identificador = p3['image_id']
                    if p3['image_id'] in dict_counter:
                        dict_counter[identificador] = dict_counter[identificador] + 1
                    else:
                        dict_counter[identificador] = 1
            

            json_my_annotations = PATH_MAP            
            total_number_elements = 0
            elements_per_frame = {}
            score_per_frame = {}

            with open(json_my_annotations) as json_result_file:
                data_result = json.load(json_result_file)
                for data in data_result:
                    if data['score']*100 > score:
                        total_number_elements = total_number_elements + 1
                        id_imagen = data['image_id']
                        if id_imagen not in elements_per_frame:
                            elements_per_frame[id_imagen] = 1
                        else:
                            elements_per_frame[id_imagen] = elements_per_frame[id_imagen]+1
                        if id_imagen not in score_per_frame:
                            score_per_frame[id_imagen]= [data['score']*100]
                        else:
                            lista = score_per_frame[id_imagen]
                            lista.append(data['score']*100)
                            score_per_frame[id_imagen] = lista

            list_elements = []
            for element in elements_per_frame.keys():
                list_elements.append(elements_per_frame[element])
          
            st_dev = statistics.pstdev(list_elements)
            json_data.append(round(sum(list_elements) / len(list_elements),3))
            json_data.append(round(st_dev,3))
            
            list_score = []
            score_finals = {}

            for key, score in score_per_frame.items():
                suma = 0
                for puntuacion in score:
                    suma = suma + puntuacion
                suma = suma / dict_counter[key]
                list_score.append(suma)
           
            st_dev2 = statistics.pstdev(list_score)
            json_data.append(round(sum(list_score) / len(list_score),3))
            json_data.append(round(st_dev2,3))
            data_final[MODELS_NAMES][SELECTED_SEQ][SEQ_DATA_NAME][int(WSizeR*10)] = json_data
        
json_pareto = json.dumps(data_final)   
PATH_EVAL_JSON = PATH_EVAL_JSON + 'OUTPUT.json'    
with open(PATH_EVAL_JSON, 'w') as outfile:
    outfile.write(json_pareto)
